### Azure의 document_intelligence를 활용하여 OCR 진행하고 polygon 그리기
Document Intelligence Studio에서 다양한 툴 사용해보고, prebuilt-read model 리소스를 활용하여 이미지를 넣으면 글자 분석 결과와 인식된 글자에 polygon을 만드는 실습 진행.
https://documentintelligence.ai.azure.com/studio?tenantid=5fb256f0-fbf2-40d2-81d5-bac1b32c419d

prebuild-read의 post 및 get requests 관련해서는 postman의 Documnet_Intelligence_Read 항목 참고하기

In [ ]:
import gradio as gr # type: ignore
import requests # type: ignore
import time
from PIL import Image, ImageDraw # type: ignore
import random
import os
from dotenv import load_dotenv

load_dotenv()
DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("DOCUMENT_INTELLIGENCE_ENDPOINT")
DOCUMENT_INTELLIGENCE_APIKEY = os.getenv("DOCUMENT_INTELLIGENCE_APIKEY")

#### - prebuild-read 관련해서 requests 관련 함수

In [ ]:
# prebuild-read: 호출하는 메소드
def request_document_intelligence(file_path):
    endpoint = DOCUMENT_INTELLIGENCE_ENDPOINT
    
    header = {
        "Ocp-Apim-Subscription-Key":DOCUMENT_INTELLIGENCE_APIKEY,
        "Content-Type":"image/png"
    }

    with open(file_path, "rb") as image:
        image_data = image.read()

    # file_path로부터 파일을 읽은 다음에, binary(바이너리) 형태로 읽어와서 request body에 넣어주면 된다. 따라서 body 필요없음.
    response = requests.post(endpoint, headers=header, data=image_data)
    # print(response.status_code, response.headers)

    # 처음에 위에서 입력 이미지 주소가 잘 들어가서, prebuild의 read에 연결이 잘 되면 202가 뜸.
    if response.status_code == 202:
        # response.headers['Operation-Location'] 위치에 결과로 나온 enpoint url이 나옴.
        result_url = response.headers['Operation-Location']
        # print(result_url)

        # 받아온 url과 기존 key를 활용해 get 응답을 받음.
        result_response = requests.get(result_url, headers={
            "Ocp-Apim-Subscription-Key": header["Ocp-Apim-Subscription-Key"]
        })
        result_response_json = result_response.json()
        result_status = result_response_json['status']

        # 상태를 계속 받아서, succeeded가 뜨면 대기 상태를 벗어남.
        while True:
            if result_status == "running":
                print(result_status)
                time.sleep(1)
                continue
            else:
                break

        # 결과 출력
        if result_status == "succeeded":
            print(result_status)
            # print(result_response_json)
            # 이거 아래가 이미지 인식한 결과
            # print(result_response_json['analyzeResult']['content'])
            return result_response_json
        else:
            return ""

#### - polygon 그리는 함수
위에서 호출받은 result_response(polygon 위치) 및 원본 이미지 파일경로를 활용해서 그리는 함수

In [ ]:
# polygon data + 입력이미지를 받아서, 입력 이미지에 그려주는 메소드.
def draw_image(file_path, result_response):
    image = Image.open(file_path)
    draw = ImageDraw.Draw(image)

    # TODOpolygon을 그려주면 끝.
    block_list = result_response['analyzeResult']['paragraphs']

    for block in block_list:
        line_color = random_color()
        polygon_list = block['boundingRegions'][0]['polygon']
        points = [(polygon_list[i], polygon_list[i+1]) for i in range(0, len(polygon_list), 2)] # 자료를 튜플형으로 변환
        draw.polygon(points, outline=line_color, width=2)

    return image

def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

#### - gradio에서 전송 버튼 클릭 동작 함수

In [ ]:
def click_send(file_path):
    result_response = request_document_intelligence(file_path)
    image = draw_image(file_path, result_response)
    return image

#### - gradio 구현부분

In [ ]:
with gr.Blocks() as demo:
    input_image = gr.Image(label="입력 이미지", type="filepath")
    send_button = gr.Button("전송")
    output_image = gr.Image(label="출력 이미지", interactive=False)

    send_button.click(click_send, inputs=[input_image], outputs=[output_image])

demo.launch()
# file_path = "C:/Users/jooeu/Desktop/git/ms-ai-school/250319_Document_Intelligence/read-resume.png"
# result_response = request_document_intelligence(file_path)
# draw_image(file_path, result_response)

running
* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


running
running
running
running
running
running
running
running
succeeded
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
runnin